In [64]:
from os import listdir
import pandas as pd
import numpy as np
from astropy.io import fits
import astropy.io.fits as pyfits
from astropy.wcs import WCS
import matplotlib.pyplot as plt
from astropy.table import Table
import pandas as pd
import math
import random as r
import scipy.stats
import os.path
import sys
sys.path.append('/home/machado.35/projects/modules')
from deprojectGalaxy import deproject
import time
import spacing_calc
import astropy.units as u

In [47]:
low_sens

['eso097-013',
 'ic1954',
 'ic5273',
 'ngc0253',
 'ngc0628',
 'ngc0685',
 'ngc1087',
 'ngc1097',
 'ngc1300',
 'ngc1317',
 'ngc1365',
 'ngc1385',
 'ngc1433',
 'ngc1511',
 'ngc1512',
 'ngc1546',
 'ngc1559',
 'ngc1566',
 'ngc1637',
 'ngc1792',
 'ngc1809',
 'ngc2090',
 'ngc2283',
 'ngc2566',
 'ngc2775',
 'ngc2835',
 'ngc2903',
 'ngc2997',
 'ngc3059',
 'ngc3137',
 'ngc3351',
 'ngc3507',
 'ngc3511',
 'ngc3521',
 'ngc3596',
 'ngc3599',
 'ngc3621',
 'ngc3626',
 'ngc3627',
 'ngc4207',
 'ngc4254',
 'ngc4293',
 'ngc4298',
 'ngc4303',
 'ngc4321',
 'ngc4424',
 'ngc4457',
 'ngc4459',
 'ngc4476',
 'ngc4477',
 'ngc4496a',
 'ngc4535',
 'ngc4536',
 'ngc4540',
 'ngc4548',
 'ngc4569',
 'ngc4571',
 'ngc4596',
 'ngc4689',
 'ngc4694',
 'ngc4731',
 'ngc4781',
 'ngc4826',
 'ngc4941',
 'ngc4945',
 'ngc4951',
 'ngc5042',
 'ngc5068',
 'ngc5128',
 'ngc5134',
 'ngc5236',
 'ngc5248',
 'ngc5530',
 'ngc5643',
 'ngc6300',
 'ngc6744',
 'ngc7496',
 'ngc7743',
 'ngc7793']

In [43]:
list(set(high_sens)-set(low_sens))

['ngc0247', 'ngc3489', 'ngc0300', 'ngc7456']

In [37]:
high_sens = listdir('/data/rubin/machado.35/phangs/catalogs/cprops_v4/homogenized_150pc_high_sens')
low_sens = listdir('/data/rubin/machado.35/phangs/catalogs/cprops_v4/homogenized_150pc_low_sens')
cov_masks = listdir('/home/machado.35/projects/masks/coverage/')
exp_masks = listdir('/home/machado.35/projects/masks/exp/')

In [38]:
for i in range(len(high_sens)):
    index = high_sens[i].index('_')
    high_sens[i] = high_sens[i][0:index]

for i in range(len(low_sens)):
    index = low_sens[i].index('_')
    low_sens[i] = low_sens[i][0:index]

for i in range(len(cov_masks)):
    index = cov_masks[i].index('_')
    cov_masks[i] = cov_masks[i][0:index]

for i in range(len(exp_masks)):
    index = exp_masks[i].index('_')
    exp_masks[i] = exp_masks[i][0:index]

In [5]:
sources = pd.read_csv('/home/machado.35/projects/intro/scripts/sources.csv')
sources = list(sources['sources'])

In [40]:
list(set(cov_masks)-set(exp_masks))

['ngc7743',
 'ngc4459',
 'ngc7456',
 'ngc5236',
 'ngc2775',
 'ngc3059',
 'ngc3239',
 'ngc3599',
 'ngc4477',
 'ngc3489',
 'ngc1809',
 'ngc4596',
 'ic5332',
 'ngc4476']

# SOURCES IN LOW SENSITIVITY CATALOG THAT ARE NEW

In [17]:
list(set(low_sens)-set(sources))

['ngc7793',
 'ngc7743',
 'ngc4459',
 'eso097-013',
 'ngc5236',
 'ngc2775',
 'ngc5128',
 'ngc3059',
 'ngc4945',
 'ngc3599',
 'ngc1809',
 'ngc4596',
 'ngc4477',
 'ngc4476']

# SOURCES IN HIGH SENSITIVITY CATALOG THAT ARE NOT IN ORIGINAL SET

In [19]:
list(set(high_sens)-set(sources))

['ngc7793',
 'ngc7743',
 'ngc4459',
 'ngc7456',
 'ngc5236',
 'ngc5128',
 'ngc0300',
 'ngc4945',
 'ngc3489',
 'ngc4596',
 'ngc0247',
 'ngc4477',
 'ngc4476']

# SOURCES IN LOW SENS THAT ARE NOT IN HIGH SENS

In [20]:
list(set(low_sens)-set(high_sens))

['ngc4689',
 'eso097-013',
 'ngc3059',
 'ngc1566',
 'ngc4540',
 'ngc2283',
 'ngc1365',
 'ngc4424',
 'ngc6300',
 'ngc5530',
 'ngc4321',
 'ngc4207',
 'ngc4571',
 'ngc2775',
 'ngc2090',
 'ngc1809',
 'ngc5248',
 'ngc1559',
 'ngc1317',
 'ngc1300',
 'ngc1512',
 'ngc1433',
 'ngc4496a',
 'ngc3599',
 'ngc2835',
 'ngc1385',
 'ngc4951',
 'ngc5042',
 'ngc2566',
 'ngc3626',
 'ngc4457',
 'ngc5134',
 'ngc4535',
 'ngc4303',
 'ngc4694',
 'ngc4293',
 'ngc1087',
 'ngc6744',
 'ngc3507',
 'ngc3596']

In [21]:
len(low_sens)

79

In [22]:
#REAL TEST
radii = [100,200,400,600,800,1000,1500]
for k in range(len(res)):
    for i in range(len(sources)):
        image = '/home/machado.35/projects/masks/exp/'+sources[i]+'_'+str(res[k])+'pc_exp_mask.fits'
        cat_fp = '/home/machado.35/projects/intro/sources/'+sources[i]+'/'+sources[i]+'_'+str(res[k])+'pc_cloud_stats.csv'
        fn = output_path+sources[i]+'_exp'+'_'+str(res[k])+'pc.csv'
        if os.path.isfile(image):
            if os.path.isfile(cat_fp):
                if os.path.isfile(fn):
                    row = int(np.where(t['name'] == sources[i])[0])
                    dist = t['dist'][row]*1e6 #from Mpc to pc
                    df = pd.read_csv(fn)
                    cat = pd.read_csv(cat_fp)
                    x = df['rRA']
                    y = df['rDEC']
                    N_tot = len(x)
                    n_it = len(cat)
                    n_loops = int(np.floor(N_tot / n_it))
                    mindist = np.zeros_like(x)*np.nan
                    mindist3 = np.zeros_like(x)*np.nan
                    mindist5 = np.zeros_like(x)*np.nan
                    radial_density_100 = np.zeros_like(x)*np.nan
                    radial_density_200 = np.zeros_like(x)*np.nan
                    radial_density_400 = np.zeros_like(x)*np.nan
                    radial_density_600 = np.zeros_like(x)*np.nan
                    radial_density_800 = np.zeros_like(x)*np.nan
                    radial_density_1000 = np.zeros_like(x)*np.nan
                    radial_density_1500 = np.zeros_like(x)*np.nan
                    for ii in np.arange(n_loops):
                        tx = x[ii*n_it:(ii+1)*n_it]
                        ty = y[ii*n_it:(ii+1)*n_it]
                        mindist[ii*n_it:(ii+1)*n_it] = spacing_calc.pair_dist_calc(x=tx,y=ty, neighbor_num=1) #1st nearest
                        mindist3[ii*n_it:(ii+1)*n_it] = spacing_calc.pair_dist_calc(x=tx,y=ty, neighbor_num=3) #3rd nearest
                        mindist5[ii*n_it:(ii+1)*n_it] = spacing_calc.pair_dist_calc(x=tx,y=ty, neighbor_num=5) #5th nearest
                        radial_density_100[ii*n_it:(ii+1)*n_it] = spacing_calc.rad_density(x=tx,y=ty, rad=radii[0], galdist=dist)
                        radial_density_200[ii*n_it:(ii+1)*n_it] = spacing_calc.rad_density(x=tx,y=ty, rad=radii[1], galdist=dist)
                        radial_density_400[ii*n_it:(ii+1)*n_it] = spacing_calc.rad_density(x=tx,y=ty, rad=radii[2], galdist=dist)
                        radial_density_600[ii*n_it:(ii+1)*n_it] = spacing_calc.rad_density(x=tx,y=ty, rad=radii[3], galdist=dist)
                        radial_density_800[ii*n_it:(ii+1)*n_it] = spacing_calc.rad_density(x=tx,y=ty, rad=radii[4], galdist=dist)
                        radial_density_1000[ii*n_it:(ii+1)*n_it] = spacing_calc.rad_density(x=tx,y=ty, rad=radii[5], galdist=dist)
                        radial_density_1500[ii*n_it:(ii+1)*n_it] = spacing_calc.rad_density(x=tx,y=ty, rad=radii[6], galdist=dist)
                    df['nn_dist'] = np.deg2rad(mindist) * dist
                    df['nn3_dist'] = np.deg2rad(mindist3) * dist
                    df['nn5_dist'] = np.deg2rad(mindist5) * dist
                    df['clouds_r'+str(radii[0])+'pc'] = radial_density_100
                    df['clouds_r'+str(radii[1])+'pc'] = radial_density_200
                    df['clouds_r'+str(radii[2])+'pc'] = radial_density_400
                    df['clouds_r'+str(radii[3])+'pc'] = radial_density_600
                    df['clouds_r'+str(radii[4])+'pc'] = radial_density_800
                    df['clouds_r'+str(radii[5])+'pc'] = radial_density_1000
                    df['clouds_r'+str(radii[6])+'pc'] = radial_density_1500
                    df.to_csv(fn)

hi


In [65]:
fn = '/data/rubin/machado.35/phangs/catalogs/cprops_v4/homogenized_150pc_high_sens/ngc3621_12m+7m+tp_co21_150pc_46mK_props.fits'
tab = Table.read(fn)
x = tab['XCTR_DEG']*u.deg
y = tab['YCTR_DEG']*u.deg

tab_loc = '/home/machado.35/projects/intro/phangs_sample_table_v1p6.fits'
t = Table.read(tab_loc)
row = int(np.where(t['name'] == 'ngc3621')[0])
galRA=t[row]['orient_ra']
galDEC=t[row]['orient_dec']
pa=t[row]['orient_posang']
incl=t[row]['orient_incl']
galDist=t[row]['dist']

center_coord = [galRA,galDEC]

In [69]:
deproject(center_coord=center_coord, incl=incl, pa=pa, ra=x, dec=y, return_offset=True)[1]

array([  28.05785886,  133.94333419,    4.6631663 ,   -3.72099101,
         -6.65650174,  -17.53905183,   21.7493016 ,  -47.94809967,
         47.82127564,  -61.41427527,   57.32750098,   52.39980823,
         50.84320725,   71.78882691,   39.97023482,   46.48213439,
         71.20064587,   61.42913093,   65.87274579,   64.85585058,
         70.32347537,   82.20326588,   87.24106447,   76.72650572,
        -85.80132101,   86.20604521,   93.08063895,   95.06911718,
         96.59812536,   96.03529732,  100.95986594, -141.35932165,
       -108.26166615,   60.23834725,  -55.7631657 ,  -74.03986002,
         11.64127951,  -44.21921208,  -51.52990826,  -59.0157823 ,
       -119.21523661,  -65.99319418,   38.60668748,  -63.74463046,
        -69.96147938, -107.31298722,  116.48333148,  178.18947766,
       -111.97972382, -127.26052852, -125.00992046,  -31.50023176,
       -147.56892273, -132.11834526,  -94.37837649,  -59.77952811,
       -122.6659016 , -105.89803662,  -67.15316551,   39.18182

In [54]:
deproject

<function deprojectGalaxy.deproject(center_coord=None, incl=<Quantity 0. deg>, pa=<Quantity 0. deg>, header=None, wcs=None, naxis=None, ra=None, dec=None, return_offset=False)>

<Quantity [-32.78031711, -32.85445298, -32.77973786, -32.77739335,
           -32.78181959, -32.7803593 , -32.78126913, -32.78168097,
           -32.78396524, -32.7858719 , -32.79822955, -32.7992457 ,
           -32.79565829, -32.80754961, -32.7922329 , -32.79339777,
           -32.80623589, -32.80235325, -32.80377717, -32.7995426 ,
           -32.80145902, -32.8119643 , -32.81487702, -32.80926247,
           -32.80517692, -32.81496852, -32.81994157, -32.8216155 ,
           -32.81669376, -32.81845327, -32.8193118 , -32.82630421,
           -32.8214607 , -32.80521418, -32.7890523 , -32.80002457,
           -32.78454927, -32.79246369, -32.79159535, -32.79317909,
           -32.82370721, -32.79455624, -32.79684183, -32.79561125,
           -32.79692149, -32.82028794, -32.8216393 , -32.84091405,
           -32.82055091, -32.82839315, -32.82959939, -32.78571659,
           -32.8317387 , -32.8309206 , -32.8111044 , -32.79892597,
           -32.82300183, -32.81886406, -32.8051059 , -32.80964